In [129]:
import pandas as pd
import numpy as np

df = pd.read_pickle('R-TXF1-1Minute-Trade-2016.pkl')

back_bar_num = 1500
forward_bar_num = 7
df['AVG'] = (df['High'] + df['Low'] + df['Close']) / 3
tmp_df = df['Open']
df['Norm Open'] = (tmp_df - df['AVG'].rolling(back_bar_num).mean()) / df['AVG'].rolling(back_bar_num).std()
tmp_df = df['High']
df['Norm High'] = (tmp_df - df['AVG'].rolling(back_bar_num).mean()) / df['AVG'].rolling(back_bar_num).std()
tmp_df = df['Low']
df['Norm Low'] = (tmp_df - df['AVG'].rolling(back_bar_num).mean()) / df['AVG'].rolling(back_bar_num).std()
tmp_df = df['Close']
df['Norm Close'] = (tmp_df - df['AVG'].rolling(back_bar_num).mean()) / df['AVG'].rolling(back_bar_num).std()
tmp_df = df['Volume']
df['Norm Volume'] = (tmp_df - tmp_df.rolling(back_bar_num).mean()) / tmp_df.rolling(back_bar_num).std()
df['Highest'] =  df['High'].rolling(forward_bar_num).max().shift(-1 * forward_bar_num)
df['Max Gain'] = df['Highest'] - df['Open'].shift(-1)
df['Lowest'] =  df['Low'].rolling(forward_bar_num).min().shift(-1 * forward_bar_num)
df['Max Loss'] = df['Lowest'] - df['Open'].shift(-1)
df['HL'] = df['High'] - df['Low']
new_df = df[back_bar_num:-1 * forward_bar_num]
new_df['class'] = 0
new_df['class'][(new_df['Max Loss'] > -20) & (new_df['Max Gain'] > 20)] = 1
# new_df['class'][(new_df['Max Loss'] > -50) & (new_df['Max Gain'] > 100)] = 2
# new_df['class'][(new_df['Max Loss'] > -50) & (new_df['Max Gain'] > 200)] = 3
# new_df['class'][(new_df['Max Loss'] > -50) & (new_df['Max Gain'] > 300)] = 4
new_df['class0'] = 0
new_df['class0'][new_df['class'] == 0] = 1
new_df['class1'] = 0
new_df['class1'][new_df['class'] == 1] = 1
new_df['class2'] = 0
new_df['class2'][new_df['class'] == 2] = 1
new_df['class3'] = 0
new_df['class3'][new_df['class'] == 3] = 1
new_df['class4'] = 0
new_df['class4'][new_df['class'] == 4] = 1
print(new_df[new_df['class'] == 0].size / new_df.size)
print(new_df.describe())
print(new_df)

/Users/cb.hsu/anaconda/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/cb.hsu/anaconda/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/cb.hsu/anaconda/envs/py3/lib/python3.6/site-packages/pandas/core/generic.py:5984: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/

0.9490162627773971
              Open          High           Low         Close        Volume  \
count  71513.00000  71513.000000  71513.000000  71513.000000  71513.000000   
mean    8743.95391   8746.249088   8741.653000   8743.962916    419.526338   
std      445.49086    445.052211    445.900316    445.471772    509.006521   
min     7513.00000   7529.000000   7503.000000   7526.000000      1.000000   
25%     8467.00000   8469.000000   8464.000000   8467.000000    122.000000   
50%     8778.00000   8780.000000   8775.000000   8777.000000    259.000000   
75%     9140.00000   9142.000000   9137.000000   9139.000000    521.000000   
max     9438.00000   9440.000000   9434.000000   9438.000000   9838.000000   

            Highest          Gain           AVG     Norm Open     Norm High  \
count  71513.000000  71513.000000  71513.000000  71513.000000  71513.000000   
mean    8750.596647     87.589963   8743.955001      0.178683      0.222518   
std      444.162865    121.916817    445.

In [130]:
num = 7
X = pd.DataFrame()
for i in range(num):
    X = pd.concat(
        [
            X,
            new_df[['Norm Open', 'Norm High', 'Norm Low', 'Norm Close', 'Norm Volume']].shift(-i),
            ],
        axis=1)

X = X[:-num]
y = new_df[['class0', 'class1']][:-num]

In [131]:
from __future__ import print_function
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train = X_train.as_matrix()

X_test0 = X_test[y_test['class0'] == 1].as_matrix()
X_test1 = X_test[y_test['class1'] == 1].as_matrix()
y_train = y_train.as_matrix()
y_test0 = y_test[y_test['class0'] == 1].as_matrix()
y_test1 = y_test[y_test['class1'] == 1].as_matrix()

X_test = X_test.as_matrix()
y_test = y_test.as_matrix()

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(53629, 35)
(53629, 2)
(17877, 35)
(17877, 2)


In [132]:
import tensorflow as tf

def add_layer(inputs, in_size, out_size, activation_function=None,):
    # add one more layer and return the output of this layer
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1,)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b,)
    return outputs

def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys})
    return result


In [133]:
# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 35]) # 28x28
ys = tf.placeholder(tf.float32, [None, 2])

# add output layer
l1 = add_layer(xs, 35, 100,  activation_function=tf.nn.softmax)
l2 = add_layer(l1, 100, 20,  activation_function=tf.nn.softmax)
prediction = add_layer(l2, 20, 2,  activation_function=tf.nn.softmax)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=ys))
learning_rate = 0.01
train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
sess = tf.Session()

init = tf.global_variables_initializer()
sess.run(init)

In [136]:
# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 35]) # 28x28
ys = tf.placeholder(tf.float32, [None, 2])

# add output layer
l1 = add_layer(xs, 35, 20,  activation_function=tf.nn.softmax)
l2 = add_layer(l1, 20, 20,  activation_function=tf.nn.softmax)
prediction = add_layer(l2, 20, 2,  activation_function=tf.nn.softmax)

logits = prediction
class_weight = tf.constant([-10000.,10000.])
scaled_logits = tf.multiply(logits, class_weight)
scaled_ys = tf.multiply(ys, class_weight)
# prediction = scaled_logits
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=scaled_logits, labels=ys))
learning_rate = 0.01
train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
sess = tf.Session()

init = tf.global_variables_initializer()
sess.run(init)

In [137]:
for i in range(400):
    batch_xs = X_train
    batch_ys = y_train
    sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys})
    if i % 50 == 0:
        acc_all = compute_accuracy(X_test, y_test)
        acc_0 = compute_accuracy(X_test0, y_test0)
        acc_1 = compute_accuracy(X_test1, y_test1)
        y_0_size = y_test0.size / 2
        y_1_size = y_test1.size / 2
        print(y_1_size*acc_1 / (y_1_size*acc_1 + y_0_size*(1-acc_0)))


0.05073686610828741
0.05088458392049718
0.050749453275761476
0.050858875977222454
0.05109652691556276
0.050953985531873726
0.05099007263859625
0.050822710951169414
